In [1]:
import os
import numpy
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import keras

/dvlscratch/SHI/users/hofman/ML/venv3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [3]:
STA = 'LPAZ'

In [5]:
#load data from oracle do pandas
#connect to udb
with open(os.path.join('/','home','hofman','.dbp.txt'), 'r') as f: password_old = f.read().strip()
import cx_Oracle
connection = cx_Oracle.connect('hofman', password_old, 'udb')

In [6]:
from datetime import datetime as dt
ts_2017 = dt(2017,1,1,0,0,0).timestamp()
ts_2018 = dt(2018,1,1,0,0,0).timestamp()
ts_2019 = dt(2019,1,1,0,0,0).timestamp()

In [42]:
query = f"select * from EVAL_QANDEF"   ## where sta='{STA}'"
df_all = pd.read_sql(query, con=connection)

In [43]:
df_all.shape

(48070, 12)

In [45]:
df_all.head()

ARID_ODB      TIME_ODB     AZI_ODB IPHASE_QANDEF IPHASE_ODB ASSOC_PHASE  \
0  130466587  1.520198e+09    6.470828             P         Pn        None   
1  130466588  1.520198e+09    5.636480             P         Pn        None   
2  130466904  1.520199e+09  154.610430             P          P        None   
3  130467276  1.520201e+09  282.437680             P          P        None   
4  130467277  1.520201e+09  168.726310             P          P        None   

  CLASS_IPHASE_QANDEF CLASS_IPHASE_ODB CLASS_PHASE_ODB  NCLS_IPHASE_QANDEF  \
0                   T             regP               N                   1   
1                   T             regP               N                   1   
2                   T                T               N                   1   
3                   T                T               N                   1   
4                   T                T               N                   1   

   NCLS_IPHASE_ODB  NCLS_PHASE_ODB  
0                2               0  
1                2               0  
2                1               0  
3                1               0  
4                1               0

In [46]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    NCLS_IPHASE_QANDEF = df_all['NCLS_IPHASE_QANDEF'].as_matrix()
    NCLS_IPHASE_ODB = df_all['NCLS_IPHASE_ODB'].as_matrix()
    NCLS_PHASE_ODB = df_all['NCLS_PHASE_ODB'].as_matrix()

In [47]:
NCLS_PHASE_ODB.min(), NCLS_PHASE_ODB.max()

(0, 3)

In [48]:
df_all[numpy.isnan(df_all['NCLS_PHASE_ODB'])]


Empty DataFrame
Columns: [ARID_ODB, TIME_ODB, AZI_ODB, IPHASE_QANDEF, IPHASE_ODB, ASSOC_PHASE, CLASS_IPHASE_QANDEF, CLASS_IPHASE_ODB, CLASS_PHASE_ODB, NCLS_IPHASE_QANDEF, NCLS_IPHASE_ODB, NCLS_PHASE_ODB]
Index: []

In [49]:
from sklearn.metrics import confusion_matrix

In [54]:
def confmat(x,y):
    C = confusion_matrix(x,y)
    diagsum = numpy.diag(C).sum()
    accuracy = diagsum/C.sum()
    print (C)
    print('Assoc accuracy: %3.2f%%' % ((numpy.diag(C)[1:].sum())/(C[:,1:].sum())*100))
    print('Overal accuracy: %3.2f%%' % (accuracy*100))
    print('N phase rate: %3.2f%%' % (C[0,1:].sum()/C[:,1:].sum()*100))

In [55]:
#confusion_matrix(y_true, y_pred)
print('OLD:')
print(confmat(NCLS_IPHASE_ODB, NCLS_PHASE_ODB))

print('NEW')
print(confmat(NCLS_IPHASE_QANDEF, NCLS_PHASE_ODB))

OLD:
[[12263   195     0    14]
 [24967  2922   129     5]
 [ 5569   501    73     3]
 [ 1364    58     0     7]]
Assoc accuracy: 76.84%
Overal accuracy: 31.76%
N phase rate: 5.35%
None
NEW
[[13095   104     0     7]
 [24902  3030   125     3]
 [ 4291   408    77     6]
 [ 1875   134     0    13]]
Assoc accuracy: 79.86%
Overal accuracy: 33.73%
N phase rate: 2.84%
None
